In [1]:
# implement a rnn encoder decoder network for seq 2 seq
import torch
from torch import nn
from torch import optim

### DATA

In [14]:
# to keep thing simple lets use one hot encoding representations for our sequential data
# we are going to split the data X, Y
data = [["i like to eat fish", "toi thit an ca"],
		["have you ate yet", "co an com chua"],
		["we are going to church tomorrow", "ngay may minh di le"]]

english = {}
idx = 0
for entry in data:
	tokens = entry[0].split(" ")
	for token in tokens:

		if token not in english:
			english[token] = idx
			idx += 1

vietnamese = {}
idx = 0
for entry in data:
	tokens = entry[1].split(" ")
	for token in tokens:

		if token not in vietnamese:
			vietnamese[token] = idx
			idx += 1

EMBEDDING_SIZE = 25
MAX_SEQ_LEN = 10


# NEED TO MODIFY THE BELOW TO GET THE SEQUENCE AND OHE FOR TEXT PER LANGUAGE
input_data = []
for entry in data:
	pair = []
	for sentence in entry:
		tokens = sentence.split(" ")

		sequence = []
		for token in tokens:
			ohe = [0]*EMBEDDING_SIZE
			ohe[lexicon[token]] = 1
			sequence.append(ohe)
		while len(sequence) < MAX_SEQ_LEN:
			ohe = [0]*EMBEDDING_SIZE
			sequence.append(ohe)

		pair.append(sequence)
	input_data.append(pair)

data = torch.Tensor(input_data) # N, 2, L, Hin
X = data[:, 0, :, :]
Y = data[:, 1, :, :]
print(X.size())

torch.Size([3, 10, 26])


### MODEL

In [ ]:
class Encoder(nn.Module):
	def __init__(self, input_size, hidden_size, num_layers=1, bidirectional=False, batch_first=True):
		super(self, Encoder).__init__()
		self.rnn = nn.RNN(input_size=input_size,
					hidden_size=hidden_size,
					num_layers=num_layers,
					bidirectional=bidirectional,
					batch_first=batch_first)

	def forward(self, x):
		output, hn = self.rnn(x)
		return output, hn

# check image on read me
# if bidirectional = True
# 	according to torch LSTM doc, hn contains the final hidden states of forward and backward
#	output contains the forward output and backward output at time step t

In [ ]:
class Decoder(nn.Module):
	def __init__(self, input_size, hidden_size, num_layers=1, bidirectional=False, batch_first=True):
		super(self, Decoder).__init__()
		self.rnn = nn.RNN(input_size=input_size,
					hidden_size=hidden_size,
					num_layers=num_layers,
					bidirectional=bidirectional,
					batch_first=batch_first)
		
		self.linear = nn.Linear(hidden_size, EMBEDDING_SIZE) # note that 